In [1]:
from fuzzywuzzy import fuzz, process
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from usefulfcns import create_metrics

In [2]:
df = pd.read_csv('/Users/jbachlombardo/Documents/Tere International/Clients/PIPD/Ad data/Lifetime Jan 12 2019/PIPD-Ads-Lifetime_190112_region.csv')

In [3]:
ppo_map = dict()
for a in process.extract('Pro-Palestine Orgs', df['Ad set name'].unique(), limit = 15, scorer = fuzz.token_sort_ratio) :
    if a[1] > 70 :
        ppo_map[a[0]] = 'Pro-Palestine Orgs'

In [4]:
pipd_ll_map = dict()
for a in process.extract('PIPD Lookalike', df['Ad set name'].unique(), limit = 15, scorer = fuzz.token_set_ratio) :
    if a[1] >= 90 :
        pipd_ll_map[a[0]] = 'US PIPD Lookalike'

In [5]:
sj_map = dict()
for a in process.extract('Social justice', df['Ad set name'].unique(), limit = 15, scorer = fuzz.token_set_ratio) :
    if a[1] > 95 :
        sj_map[a[0]] = 'US Social Justice'

In [6]:
pipd_eng_map = {'PIPD Engagement': 'US PIPD Engagement', 'PIPD engagement': 'US PIPD Engagement', 'PIPD Engagement US': 'US PIPD Engagement'}

In [7]:
df['Ad set name'].value_counts().loc[sj_map.keys()].sum() + df['Ad set name'].value_counts().loc[pipd_eng_map.keys()].sum() + df['Ad set name'].value_counts().loc[ppo_map.keys()].sum() + df['Ad set name'].value_counts().loc[pipd_ll_map.keys()].sum()

53091

In [8]:
mapper = {**ppo_map, **pipd_ll_map, **pipd_eng_map, **sj_map}

In [9]:
df['Grouped US set'] = df['Ad set name'].map(mapper)

In [10]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming']

In [11]:
df['Grouped US set'] = df['Grouped US set'].fillna('Other')

In [12]:
df = df[df['Region'].isin(states)]

In [13]:
state_level = create_metrics(df, level = ['Grouped US set', 'Region'])

In [22]:
state_level.columns

Index(['Reach', 'Impressions', 'Post engagement', 'Post comments',
       'Post shares', 'Link clicks', 'Amount spent (USD)', 'Cost per reach',
       'Cost per impressions', 'Cost per post engagement',
       'Cost per post comments', 'Cost per post shares',
       'Cost per link clicks', 'Cost per quality engagement',
       'Engagement rate'],
      dtype='object')

In [39]:
lc = create_metrics(df[df['Objective'] == 'Traffic'], level = ['Grouped US set', 'Region'])
eng = create_metrics(df[df['Objective'] == 'Engagement'], level = ['Grouped US set', 'Region'])

In [40]:
cpl = pd.pivot_table(lc, values = 'Cost per link clicks', index = 'Region', columns = 'Grouped US set')
cpl['Top performing'] = cpl.idxmin(axis = 1)

In [41]:
cpe = pd.pivot_table(eng, values = 'Cost per post engagement', index = 'Region', columns = 'Grouped US set')
cpe['Top performing'] = cpe.idxmin(axis = 1)

In [36]:
cpqe = pd.pivot_table(state_level, values = 'Cost per quality engagement', index = 'Region', columns = 'Grouped US set')
cpqe['Top performing'] = cpqe.idxmin(axis = 1)

In [44]:
lc['Clickthrough'] = lc['Link clicks'] / lc['Impressions']

In [48]:
lcc = pd.pivot_table(lc, values = 'Clickthrough', index = 'Region', columns = 'Grouped US set')
lcc['Top performing'] = lcc.idxmax(axis = 1)

In [50]:
engr = pd.pivot_table(eng, values = 'Engagement rate', index = 'Region', columns = 'Grouped US set')
engr['Top performing'] = engr.idxmax(axis = 1)

In [53]:
eng['Qual engage rate'] = eng[['Post comments', 'Post shares', 'Link clicks']].sum(axis = 1) / eng['Impressions']

In [55]:
qualengr = pd.pivot_table(eng, values = 'Qual engage rate', index = 'Region', columns = 'Grouped US set')
qualengr['Top performing'] = qualengr.idxmax(axis = 1)

In [59]:
writer = pd.ExcelWriter('/Users/jbachlombardo/Documents/Tere International/Clients/PIPD/Ad data/Lifetime Jan 12 2019/State level US.xlsx')
state_level.to_excel(writer, sheet_name = 'Raw totals')
cpl.to_excel(writer, sheet_name = 'Cost per link click')
cpe.to_excel(writer, sheet_name = 'Cost per post engagement')
cpqe.to_excel(writer, sheet_name = 'Cost per quality engagement')
lcc.to_excel(writer, sheet_name = 'Clickthrough rate (Traffic)')
engr.to_excel(writer, sheet_name = 'Engagement rate (Eng)')
qualengr.to_excel(writer, sheet_name = 'Quality engagement rate (Eng)')
writer.close()